Yelp es un sitio donde se pueden dejar reseñas y tips sobre negocios de todo tipo, muy utilizado en Estados Unidos. Los csvs son:

* businesses.csv
  * business_id
  * name
  * address
  * city
  * state
  * postal_code
  * latitude
  * longitude
  * stars: float, star rating, rounded to half-stars
  * review_count
  * is_open
  * attributes: un json con atributos variados
  * categories: lista de categorías en json a las que pertenece el negocio
  * hours: diccionario json con horarios día por día en los que abre
* review.csv
  * review_id
  * user_id
  * business_id
  * stars: entero entre 1 y 5
  * date
  * text: texto de la review
  * useful: cantidad de gente que la voto útil
  * funny: cantidad de gente que la voto divertida
  * cool: cantidad de gente que la votó como cool
* user.csv
  * user_id
  * name
  * review_count
  * yelping_since: fecha de registro
  * friends: lista en json de user ids que son amigos
  * useful: cantidad de votos “useful” que recibió el usuario
  * funny: cantidad de votos “funny” que recibió el usuario
  * cool: cantidad de votos “cool” que recibió el usuario
  * fans: cantidad de fans que tiene el usuario
  * elite: lista json con los años que el usuario fue élite
  * average_stars: promedio de rating de todas sus reviews
  * compliment_*: la cantidad de cumplidos de tipo * que recibió el usuario
* checkins.csv: Las visitas que los usuarios registran a los lugares
  * business_id
  * date: lista de fechas separadas por coma de los chekins que recibió el negocio
* tip.csv: tips que los usuarios dan sobre lugares
  * text
  * date
  * compliment_count: cumplidos que recibió el tip
  * business_id
  * user_id

## S40. Para las reviews y tips que contienen la palabra “fuck”:
- Para las reviews, ¿cuál es el promedio del score?
- ¿Cuál es el usuario que más veces usó la palabra en ambas tablas?
- ¿Cuál es el negocio que más veces recibió la palabra en ambas tablas?
- ¿Cuál es el negocio de más de 3 reviews con mayor ratio de textos que contienen “fuck”?
- Para los negocios con textos que tienen “fuck” correlacione el rating promedio del negocio con su ratio de “fuck” en textos.

### Installs e imports

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 61.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c18b2bee17cb55aed5c581e8fdcf692a2d26380200f218199d81ffae39f3056b
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 36.5 MB of archives.
After t

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

### Resolución

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType

rev_schema = StructType([StructField("review_id", StringType(), True),
                         StructField("user_id", StringType(), True),
                         StructField("business_id", StringType(), True),
                         StructField("stars", FloatType(), True),
                         StructField("useful", IntegerType(), True),
                         StructField("funny", IntegerType(), True),
                         StructField("cool", IntegerType(), True),
                         StructField("text", StringType(), True),
                         StructField("date", StringType(), True)])

tip_schema = StructType([ StructField("user_id", StringType(), True),
                          StructField("business_id", StringType(), True),
                          StructField("text", StringType(), True),
                          StructField("date", StringType(), True),
                          StructField("compliment_count", IntegerType(), True)])

sqlContext = SQLContext(sc)
rev_df = sqlContext.read.csv('/content/drive/MyDrive/Datos/TP2/review.csv', header=True, multiLine=True, schema=rev_schema)
rev_rdd = rev_df.rdd
df = sqlContext.read.csv('/content/drive/MyDrive/Datos/TP2/tip.csv', header=True, multiLine=True, schema=tip_schema)
tip_rdd = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Filtro las reviews y tips que continen "fuck" y los campos text y stars no están vacíos:

In [ ]:
rev_f = rev_rdd.filter(lambda x: x.stars and x.text and "fuck" in x.text).cache()
tip_f = tip_rdd.filter(lambda x: x.text and "fuck" in x.text).cache()

In [ ]:
rev_f.first()

Row(review_id='jF3ZlgxpM88UjKymEfSJmQ', user_id='CtRP-ARklDGTJ8aVji47Ug', business_id='ompDR5sUDpoI6gnTldmneQ', stars=2.0, useful=None, funny=None, cool=None, text='"This was my very 1st time in New Orleans. It\'s been an amazing experience and so far, every single place I\'ve eaten at has been AMAZING. So... I took a chance with The Praline Connection and walked in. Why? The ambiance and the restaurant, overall, looked nice. \nHowever... I would NEVER return to this place again, nor would I Ever RECOMMEND it to Anyone. \n\nAPPETIZERS:\nI ordered the BBQ Oysters and..wow.. fucking wow...1st of all the oysters not in their shells. Instead they placed in a soup cup filled with  ""BBQ sauce"" that tasted like cheap Ketchup and Chef Boyarde! Disgusting! The oysters themselves were gooey.', date=None)

In [ ]:
tip_f.first()

Row(user_id='-uM1ujzuJed-2QmFS-6Vog', business_id='U7HUYQAUzjBraAi_pEsHIA', text='Rip off. They fucked up my order and their prices are too high.', date='2016-11-24 02:19:58', compliment_count=0)

#### Para las reviews, ¿cuál es el promedio del score?

Para sumar la cantidad de estrellas totales, transformo el rdd a algo de tipo (clave, valor) donde todas las entradas tienen la misma clave y el valor es el numero de estrellas, para luego sumar los valores de la clave:

In [ ]:
avg = rev_f.map(lambda x: (1, x.stars)).reduceByKey(lambda x,y: x+y).first()[1] / rev_f.count()
avg

1.9804813340913399

El promedio de las reviews que contienen la palabra fuck es de 1.98 stars aproximadamente.

#### ¿Cuál es el usuario que más veces usó la palabra en ambas tablas?

Creo un rdd de tipo (clave, valor) donde la clave es el id del usuario y el valor la cantidad de "fuck" que contiene la review o tip:

In [ ]:
user_rev_f = rev_f.map(lambda x: (x.user_id, x.text.count("fuck")))
user_tip_f = tip_f.map(lambda x: (x.user_id, x.text.count("fuck")))
user_total_f = user_rev_f.union(user_tip_f)
user_total_f.reduceByKey(lambda x,y: x+y).reduce(lambda x,y: x if x[1] > y[1] else y)

('oR5az_eNCnfN7e49H3ONhg', 137)

El usuario con uid 'oR5az_eNCnfN7e49H3ONhg' es el que mas veces utilizo la palabra "fuck" con un total de 137 veces.

#### ¿Cuál es el negocio que más veces recibió la palabra en ambas tablas?

Creo un rdd de tipo (clave, valor) donde la clave es el id del negocio y el valor la cantidad de "fuck" que contiene la review o tip:

In [ ]:
business_rev_f = rev_f.map(lambda x: (x.business_id, x.text.count("fuck")))
business_tip_f = tip_f.map(lambda x: (x.business_id, x.text.count("fuck")))
cant_f_business = business_rev_f.union(business_tip_f).reduceByKey(lambda x,y: x+y).cache()
cant_f_business.reduce(lambda x,y: x if x[1] > y[1] else y)

('ytynqOUb3hjKeJfRj5Tshw', 10)

El negocio con business id 'ytynqOUb3hjKeJfRj5Tshw' es el que mas veces recibio la palabra fuck con un total de 10 veces.

#### ¿Cuál es el negocio de más de 3 reviews con mayor ratio de textos que contienen “fuck”?

Cuento la cantidad de reviews totales que tiene cada negocio y me quedo con los que tengan mas de 3:

In [ ]:
cant_reviews_business = rev_rdd.map(lambda x: (x.business_id, 1)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1] > 3)

Calculo los ratios y obtengo el mayor:

In [ ]:
business_ratio = cant_reviews_business.join(cant_f_business).map(lambda x: (x[0], x[1][1] / x[1][0])).cache()
business_ratio.reduce(lambda x,y: x if x[1] > y[1] else y)

('SeLZmaxWKWYPuHM87kNp1w', 1.4)

El negocio con business id 'SeLZmaxWKWYPuHM87kNp1w' es el que tiene un mayor ratio de palabras fuck por review, que es 1.4.

#### Para los negocios con textos que tienen “fuck” correlacione el rating promedio del negocio con su ratio de “fuck” en textos.

In [ ]:
bus_df = sqlContext.read.csv('/content/drive/MyDrive/Datos/TP2/business.csv', header=True, multiLine=True, inferSchema=True)
bus_rdd = bus_df.rdd

Obtengo la cantidad de stars de cada negocio y los uno con su ratio de "fuck" en textos, calculados en el item anterior:

In [ ]:
business_stars = bus_rdd.map(lambda x: (x.business_id, x.stars))
business_data = business_ratio.join(business_stars)
business_data.first()

('nFjk0xVI9fNiVN__5g-m8Q', (0.014760147601476014, 4.0))

Calculo la correlacion:

In [ ]:
import numpy as np

def correlation(rdd, pos1, pos2):
  """ Calcula la correlacion entre dos posiciones del value de un rdd de tipo (key, (value1, value2, ...)) """
  a = rdd.map(lambda x: x[1][pos1] * x[1][pos2]).reduce(lambda x,y: x + y)
  b = rdd.map(lambda x: x[1][pos1] * x[1][pos1]).reduce(lambda x,y: x + y) * rdd.map(lambda x: x[1][pos2] * x[1][pos2]).reduce(lambda x,y: x + y)
  return a/np.sqrt(b)

correlation(business_data, 0, 1)

0.4508260982265326

La correlación entre el rating promedio del negocio con su ratio de “fuck” en textos es 0.45 aproximadamente.